![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg)

<center>
<h1><font size="+3">GSFC Python Bootcamp</font></h1>
</center>

---

<CENTER>
<H1 style="color:red">
An Introduction to h5py
</H1>
</CENTER>

In [ ]:
from __future__ import print_function

# <font color='red'> Useful References </font>

* <A HREF="https://buildmedia.readthedocs.org/media/pdf/h5py/latest/h5py.pdf">h5py Documentation</A>
* <A HREF="https://www.christopherlovell.co.uk/blog/2016/04/27/h5py-intro.html">h5py: reading and writing HDF5 files in Python</A>
* <A HREF="https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/">How to use HDF5 files in Python</A>

## <font color="red"> What we will cover </font>
* Opening a file
* Dimension
* Variables
* Attributes
* Writing data
* Creating groups
* Reading data

## <font color='red'> What is h5py?</font>

* An HDF5 file is a container for two kinds of objects: 
   1. **Datasets**:, Array-like collections of data.
   2. **Groups**: Folder-like containers that hold datasets and other groups.
* h5py is the Python interface to the HDF5.
* Relies on NumPy arrays.
* Within h5py, HDF5 groups work like dictionaries, and datasets work like NumPy arrays.

In [ ]:
import os
import datetime as dt
import six
import numpy as np
import h5py

#### <font color='red'> Opening a netCDF File</font>

In [ ]:
hdFileName = 'sample_hdf5.h5'
modeType   = 'w'
hdfid = h5py.File(hdFileName, modeType)

* `modeType`: 'w', 'r+', 'r', or 'a'

In [ ]:
print(hdfid)

#### <font color='red'> Data Compression </font>
* When saving data, you may opt for compressing it using different algorithms. 
* h5py supports a few compression filters such as GZIP, LZF, and SZIP. 
* When using one of the compression filters, the data will be processed on its way to the disk and it will be decompressed when reading it. 
* When readding data, the underlying HDF5 library automatically extracts the data from the compressed datasets with the appropriate algorithm.  

In [ ]:
# gzip compression flag
comp = 9

* Specify the data type to optimize space
* Set data compression if needed.

#### <font color='red'> Creating Dimensions in a HDF5 File</font>

In [ ]:
# Latitude    
lat = np.arange(-90,91,2.0)
dset = hdfid.require_dataset('lat', 
                            shape=lat.shape, 
                            dtype=np.float32, 
                            compression="gzip", compression_opts=comp)
dset[...] = lat
dset.attrs['name'] = 'latitude'
dset.attrs['units'] = 'degrees north'

# Longitude
lon = np.arange(-180,180,2.5)
dset = hdfid.require_dataset('lon', 
                            shape=lon.shape, 
                            dtype=np.float32, 
                            compression="gzip", compression_opts=comp)
dset[...] = lon
dset.attrs['name'] = 'longitude'
dset.attrs['units'] = 'degrees east'

# Vertical levels
lev = np.arange(0,72,1)
dset = hdfid.require_dataset('lev', 
                            shape=lev.shape, 
                            dtype=np.int, 
                            compression="gzip", compression_opts=comp)
dset[...] = lev
dset.attrs.update({'name': 'vertical levels',
                   'units': 'hPa'})

# Time (note the unlimited dimension)
time = np.arange(0,1,1)
dset = hdfid.require_dataset('time', 
                             shape=time.shape, 
                             maxshape=(None), 
                             dtype=np.float32, compression=comp)
dset[...] = time
dset.attrs['name'] = 'time'
dset.attrs['units'] = 'hours since 2013-01-01 00:00:00.0'
dset.attrs['calendar'] = 'gregorian'

#### <font color='red'>Create Variables</font>

In [ ]:
nrecs = 5
arr = np.zeros((nrecs,lev.size,lat.size,lon.size))
arr[0:nrecs,:,:,:] = 300*np.random.uniform(
                                size=(nrecs,lev.size,lat.size,lon.size))
dset = hdfid.require_dataset('temp', shape=arr.shape, 
                                 dtype=np.float32, compression=comp)
dset[...] = arr
dset.attrs['name'] = 'temperature'
dset.attrs['units'] = 'K'

We can also use the `create_dataset` method:

In [ ]:
arr2 = np.zeros((lat.size,lon.size))
arr2[:,:] = np.random.random(size=(lat.size,lon.size))
landfrac = hdfid.create_dataset('landfrac', data=arr2, dtype=np.float32)
landfrac.attrs['name'] = 'Fraction of land'
landfrac.attrs['units'] = '1'

#### <font color='red'>Adding Global Attributes</font>

In [ ]:
hdfid.attrs['Description'] = 'Sample HDF5 file'
hdfid.attrs['History']     = 'Created for the h5py Tutorial'
hdfid.attrs['Source']      = 'NASA GSFC'
hdfid.attrs['HDF5_Version'] = six.u(h5py.version.hdf5_version)
hdfid.attrs['h5py_version'] = six.u(h5py.version.version)

In [ ]:
glob_attr = {'Date': dt.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"), 
            'User': 'ASTG',}
hdfid.attrs.update(glob_attr)

You can also use the formulation based on `json` to write metadata in the file.

In [ ]:
import json

metadata = {'Note': 'Another way to add metadata in file', 
            'OS': os.name,}
m = hdfid.create_dataset('metadata', data=json.dumps(metadata))

#### <font color='red'>Printing File Attributes</font>

In [ ]:
for k in hdfid.attrs.keys():
    print('{} => {}'.format(k, hdfid.attrs[k]))

In [ ]:
metadata_read = json.loads(hdfid['metadata'][()])
for k in metadata_read:
    print('{} => {}'.format(k, metadata_read[k]))

#### <font color='red'>List all Variables</font>

In [ ]:
for item in hdfid:
    print(item)

#### <font color='red'>Create Groups</font>
* We can organize data in hierarchical groups, which are analogous to directories in a filesystem. 
* Groups serve as containers for variables, dimensions and attributes, as well as other groups.

In [ ]:
gpData2D = hdfid.create_group('2D_Data')
gpData2D.attrs['Description'] = "Group for 2D variables"
gpData2D.attrs['Sub groups'] = "Land and Sea"

sgpLand  = gpData2D.create_group('2D_Land')
sgpSea   = gpData2D.create_group('2D_Sea')

gpData3D = hdfid.create_group('3D_Data')
gpData3D.attrs['Description'] = "Group for 2D variables"

In [ ]:
# List all variables and top groups
for item in hdfid:
    print(item)

In [ ]:
print('Parent of "2D_Land" is {}'.format(sgpLand.parent))
print('Parent of "3D_Data" is {}'.format(gpData3D.parent))

In [ ]:
for j in gpData2D.attrs.keys():
    print('{} => {}'.format(j, gpData2D.attrs[j]))

In [ ]:
for item in gpData2D:
    print(item)

In [ ]:
# Print all top variables and top groups in a file
for key in hdfid.keys():
    print(key)

The `visit()` method allows to list all the nested groups.

In [ ]:
def get_all(name):
    print(name)

In [ ]:
hdfid.visit(get_all)

The `visititems()` method allows you to view all items in the file.

In [ ]:
results=[]
def get_all_objs(name, obj):
    attr = list(obj.attrs.items())
    print(name, obj, attr)
    x = obj.attrs.get('x',None)
    if x: 
        results.append((name, x))

In [ ]:
hdfid.visititems(get_all_objs)

In [ ]:
print(results)

##### Add variable to a group

In [ ]:
arr2 = np.zeros((nrecs,lat.size,lon.size))
arr2[0:nrecs,:,:] = 300*np.random.uniform(
                                size=(nrecs,lat.size,lon.size))
surf_pres = sgpLand.create_dataset('sp', data=arr2)
surf_pres.attrs['name'] = 'Surface Pressure'
surf_pres.attrs['units'] = 'Pa'

In [ ]:
temp = gpData3D.create_dataset('temp', data=arr)
temp.attrs['name'] = 'temperature (3D)'
temp.attrs['units'] = 'K'

In [ ]:
print(hdfid["3D_Data"])

In [ ]:
print(hdfid["2D_Data/2D_Land"])

In [ ]:
for j in hdfid.attrs.keys():
    print('{} => {}'.format(j, hdfid.attrs[j]))

In [ ]:
for j in gpData2D.attrs.keys():
    print('{} => {}'.format(j, gpData2D.attrs[j]))

#### <font color='red'>Close the file</font>

In [ ]:
hdfid.close()

### <font color='red'>Reading a HDF5 File</font>

In [ ]:
with h5py.File('sample_hdf5.h5', 'r') as hdfid:
     print(hdfid.keys())

     lev  = hdfid['lev'].value
     lat  = hdfid['lat'].value
     lon  = hdfid['lon'].value
     time = hdfid['time'].value

     temp1 = hdfid['temp'].value
     print(temp1[0,0,0,0], temp1[4,6,7,15])

     temp2 = hdfid['3D_Data']['temp'].value
     print(temp2[0,0,0,0], temp2[4,6,7,15])     
        
     surf_pres = hdfid['2D_Data/2D_Land']['sp'].value

In [ ]:
print(lon)

In [ ]:
print(lat)

In [ ]:
print(temp1.shape)
print(np.min(temp1), np.max(temp1))

In [ ]:
print(temp2.shape)
print(np.min(temp2), np.max(temp2))

In [ ]:
print(surf_pres.shape)
print(np.min(surf_pres), np.max(surf_pres))

### <font color='red'>Updating a Variable in an Existing HDF5 File</font>

In [ ]:
with h5py.File(hdFileName, mode='a') as hdfid:
     temp = hdfid['temp'].value
     data = temp[:]
     data = 1.1*data + 100.0
     temp[:] = data

In [ ]:
print(temp.shape)
print(np.min(temp), np.max(temp))

### <font color='red'>Resizing Datasets</font>
* Datasets can be resized once created up to a maximum size. 
* You specify this maximum size when creating the dataset, via the keyword `maxshape`.

In [ ]:
with h5py.File('resize_dataset.h5', 'w') as hdfid:
    d = hdfid.create_dataset('dataset', (100, ),  maxshape=(500, ))
    d[:100] = np.random.randn(100)
    d.resize((200,))
    d[100:200] = np.random.randn(100)

with h5py.File('resize_dataset.h5', 'r') as hdfid:
    dset = hdfid['dataset']
    print(dset[99])
    print(dset[199])

* You can also resize the dataset at a later stage, don't need to do it in the same session when you created the file. 

In [ ]:
with h5py.File('resize_dataset.h5', 'a') as hdfid:
    dset = hdfid['dataset']
    dset.resize((300,))
    dset[:200] = 0
    dset[200:300] = np.random.randn(100)

with h5py.File('resize_dataset.h5', 'r') as hdfid:
    dset = hdfid['dataset']
    print(dset[99])
    print(dset[199])
    print(dset[299])